In [1]:
import os
import sys
import glob
import logging

from grobid_client.grobid_client import GrobidClient

In [2]:
# Set input and output directories
input_dir = "C:/Users/tinot/OneDrive/Documents/Grobid"
output_dir = "C:/Users/tinot/OneDrive/Documents/Grobid/Output"

In [3]:
# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
# Initialize Grobid client in the config.json file
client = GrobidClient(config_path="./config.json")

GROBID server is up and running


In [5]:
# Set up logging
logging.basicConfig(filename='pdf_extraction.log', level=logging.ERROR, format='%(asctime)s %(levelname)s: %(message)s')

# Save Extraction as XML

In [6]:
# Process each PDF in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.xml")

        try:
            # Process the PDF using Grobid
            # Try processFullText only
            result = client.process("processFulltextDocument", pdf_path, output=output_dir, consolidate_citations=True, tei_coordinates=True, force=True)


            # Check if the result is not None
            if result is not None:
                # Save the output XML to the specified path
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write(result)

                print(f"Successfully processed {filename}")
            else:
                # If Grobid returned None, reprocess the PDF
                print(f"Error processing {filename}: Grobid returned None. Retrying...")
                result = client.process("processFulltextDocument", pdf_path, output=output_dir, consolidate_citations=True, tei_coordinates=True, force=True)
                print(f"Error processing {filename}: Failed to retry")
                
                # Check if the result is not None
                if result is not None:
                    # Save the output XML to the specified path
                    with open(output_path, "w", encoding="utf-8") as f:
                        f.write(result)

                    print(f"Successfully processed {filename} (retried)")
                else:
                    logging.error(f"Error processing {filename}: Grobid returned None (retried)")

        except Exception as e:
            logging.error(f"Error processing {filename}: {e}")
            continue

Error processing endtoend.pdf: Grobid returned None. Retrying...
Error processing endtoend.pdf: Failed to retry
Error processing european-IXP.pdf: Grobid returned None. Retrying...
Error processing european-IXP.pdf: Failed to retry
Error processing fiftygiga.pdf: Grobid returned None. Retrying...
Error processing fiftygiga.pdf: Failed to retry


# Save Extraction as .TXT

In [7]:
# Process PDFs in input directory
for pdf_file in glob.glob(os.path.join(input_dir, '*.pdf')):
    try:
        # Extract full text from PDF
        result = client.process("processFulltextDocument", pdf_file, output=output_dir, consolidate_citations=True, tei_coordinates=True, force=True)

        # Check if result is not None
        if result is not None:
            tei_xml = result['tei']

            # Extract full text from TEI XML
            full_text = client.getFullTextFromTEI(tei_xml)

            # Write full text to output file
            output_file = os.path.join(output_dir, os.path.basename(pdf_file).replace('.pdf', '.txt'))
            with open(output_file, 'w') as f:
                f.write(full_text)

            # Print success message
            print(f'Processed {pdf_file} successfully. Output saved to {output_file}')

        else:
            # Print error message
            print(f'Error processing {pdf_file}: Grobid returned None. Skipping...', file=sys.stderr)

    except Exception as e:
        # Print error message
        print(f'Error processing {pdf_file}: {e}. Skipping...', file=sys.stderr)
        continue

Error processing C:/Users/tinot/OneDrive/Documents/Grobid\endtoend.pdf: Grobid returned None. Skipping...
Error processing C:/Users/tinot/OneDrive/Documents/Grobid\european-IXP.pdf: Grobid returned None. Skipping...
Error processing C:/Users/tinot/OneDrive/Documents/Grobid\fiftygiga.pdf: Grobid returned None. Skipping...
